### Initialize openai
* 1. Provide your api_key
* 2. Provide the api url
* You can setup your openai keys here: https://platform.openai.com/settings/organization/billing/overview

In [1]:
import openai

api_key = "Your_API_Keys_Here"
api_url = "https://api.openai.com/v1"
openai.api_key = api_key
api_base = api_url
openai.base_url = api_base

# # Provide the api key through the provided protal
client = openai.OpenAI(api_key=openai.api_key, base_url=api_base)

# import os
# os.environ["OPENAI_API_KEY"] = openai.api_key

### utils function
* 1. def show_popup: show the user query and gpt's reponse in a popup window

In [2]:
tkinter_enabled = True

try:
    import tkinter as tk
    from tkinter import Toplevel, Label, Canvas, Frame, Scrollbar

    def show_popup(query, response, PE_GPT=False):
        # Create the main application window
        root = tk.Tk()
        root.withdraw()  # Hide the main window as we only need the popup

        # Create the popup window
        popup = Toplevel(root)
        popup.title("Scrollable Popup Window")
        popup.geometry("1000x600")  # Set size of the popup window
        popup.wm_attributes("-topmost", True)  # Keep the popup on top

        # Create a frame for the scrollable content
        canvas = Canvas(popup)
        scrollbar = Scrollbar(popup, orient="vertical", command=canvas.yview)
        scrollable_frame = Frame(canvas)

        # Configure the canvas
        scrollable_frame.bind(
            "<Configure>",
            lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
        )
        canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
        canvas.configure(yscrollcommand=scrollbar.set)

        # Place the canvas and scrollbar
        canvas.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")

        text = f"User: {query}\n\n"
        if PE_GPT:
            text += f"PE-GPT: {response}"
        else:
            text += f"GPT-4.0: {response}"
        label = Label(scrollable_frame, text=text, font=("Arial", 18), wraplength=900, padx=20, pady=20)
        label.pack()

        # Add a button to close the popup and terminate the program
        def close_popup():
            popup.destroy()  # Close the popup
            root.quit()      # Exit the main loop

        # Add a button to close the popup
        close_button = tk.Button(popup, text="Close", command=close_popup, font=("Arial", 16))
        close_button.pack(pady=10)
        # Keep the popup running
        root.mainloop()
        root.destroy()  # Ensure the Tkinter process is terminated after the mainloop
except Exception as e:
    print(f"You got the error when importing tkinter: {e}.")
    print("You don't have to use the tkinter to demonstrate RAG and FunctionTool. It's just for better visualization.")
    tkinter_enabled = False


## Storyline of the Code Demo
* RAG
    * Stage 1. Modulation recommendation based on user request for performance metrics
* FunctionTool
    * Stage 2. Optimize the parameters (phase shifts) of the recommended modulation
    * Stage 3. Validate the optimized modulation in simulation

### Comparison: General-Purpose GPT-4 without RAG

In [3]:
from llama_index.llms.openai import OpenAI
from llama_index.core.llms import ChatMessage, MessageRole


llm_model = "gpt-4o-mini" # "gpt-4-0125-preview" # "gpt-3.5-turbo" 
llm = OpenAI(model=llm_model)

# Query 1: Modulation Recommendation
query = "Q1: I am looking for wide ZVS range with easy implementation. What modulation should I use for DAB converters?"
# Query 2: Information Retrival
# query = "Q2: For DAB converters, what are the controllable parameters in the 5-DoF strategy? Please elaborate"


msg = [ChatMessage(content=query, role=MessageRole.USER)]

# Chat and show
try:
    response = llm.chat(msg)
except Exception as e:
    print(f"An error occured: {e}. Check your API setup.")
    
if tkinter_enabled:
    show_popup(query, response)
else:
    print("*"*81)
    print("The user query is given below:")
    print(query)
    print("*"*81)
    print("The response is given below:")
    print(response)

## Stage 1: PE-GPT --- PE-Tailored GPT-4 with RAG

In [4]:
import os
import sys

import openai
from llama_index.core import VectorStoreIndex, ServiceContext, SimpleDirectoryReader, Document
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.llms.openai import OpenAI
from llama_index.core.node_parser import SimpleNodeParser

In [5]:
def rag_load(database_folder, llm_model, 
              temperature=None, chunk_size=None, system_prompt=None):
    """
        This function is the retrieval-augmented generation (RAG) for LLM
    """
    
    if chunk_size is None: chunk_size = 1024
    if temperature is None: temperature = 0.0
    
    
    llm = OpenAI(model=llm_model, temperature=temperature, 
                 system_prompt=system_prompt)
    
    # Step 0. Ingest/load documents
    docs = SimpleDirectoryReader(database_folder).load_data()
    # Step 1. Chunking
    node_parser = SimpleNodeParser.from_defaults(chunk_size=chunk_size)
    # create nodes 
    nodes = node_parser.get_nodes_from_documents(docs)

    # Step 2. Embedding / vectorizing, create vector database for your document 
    index = VectorStoreIndex(nodes, llm_predictor=llm)

    return index


In [6]:
temperature, chunk_size, top_k = 0.0, 1024, 7

system_prompt = """
You are PE-GPT. Please answer the user's request based on the 
documents I provided and your own knowledge. 
"""
print(system_prompt)


You are PE-GPT. Please answer the user's request based on the 
documents I provided and your own knowledge. 



In [7]:
llm_model = "gpt-4o-mini" # "gpt-4-0125-preview" # "gpt-3.5-turbo" 
index = rag_load("../core/knowledge/kb/database1", llm_model, temperature=temperature,
                 chunk_size=chunk_size, system_prompt=system_prompt)

# create a chat_engine from the created vector database
chat_engine = index.as_chat_engine(chat_mode="context", similarity_top_k=top_k)

In [8]:
chat_prompt = f"""
---------------------
System prompt is below:
{system_prompt}
---------------------
Query:
"""
# Query 1: Modulation Recommendation
query = "Q1: I am looking for wide ZVS range with easy implementation. What modulation should I use for DAB converters?"

# Query 2: Information Retrival
# query = "Q2: For DAB converters, what are the controllable parameters in the 5-DoF strategy? Please elaborate"

# Query 3: Compare the Response before and after Adding the Knowledge of PE-GPT's Initiative
# query = "Q3: What's the initiative of PE-GPT?"


chat_prompt = chat_prompt + query

print(chat_prompt)


---------------------
System prompt is below:

You are PE-GPT. Please answer the user's request based on the 
documents I provided and your own knowledge. 

---------------------
Query:
Q1: I am looking for wide ZVS range with easy implementation. What modulation should I use for DAB converters?


In [9]:
# Step 3 and Step 4: Retrieve and Query
try:
    response = chat_engine.chat(chat_prompt)
except Exception as e:
    print(f"An error occured: {e}. Check your API setup.")
    
if tkinter_enabled:
    show_popup(query, response)
else:
    print("*"*81)
    print("The user query is given below:")
    print(query)
    print("*"*81)
    print("The response is given below:")
    print(response)

### Stage 2 and Stage 3. FunctionTool
* FunctionTool 1. Conduct design tasks (e.g., optimize of modulation parameters for DAB converters)
* FunctionTool 2. Invoke plecs simulation (e.g., pass operating conditions Vin, Vout, PL and designed inner phase shift angles to simulation)
* FunctionTool 3. Invoke multi-physics modeling with PINN (Dummy)

In [10]:
import os
from typing import List
from llama_index.core.tools import FunctionTool
from llama_index.agent.openai import OpenAIAgent
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.llms.openai import OpenAI
cwd = os.getcwd()

# change the working directory to the source codes of PE-GPT
# please adjust accordingly if you want to try out
func_folder = ".." # go back to the main directory of PE-GPT
os.chdir(func_folder)
from core.simulation.load_plecs import open_plecs, PlecsThread
from core.optim.optimizers import optimize_mod_dab
os.chdir(cwd)


2025-03-17 18:14:37.307 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`


In [11]:
def conduct_design_tool(Vin: float, Vout: float, PL: float) -> str:
    """
        Design Task 1: Optimize modulation parameters of DAB converters under EPS modulation,
        requires three args from user query: 
        1. an input voltage Vin (float, unit in Volt);
        2. an output voltage Vout (float, unit in Volt);
        3. a load power PL (float, unit in W);
    """
    dab_params.Vin = Vin
    dab_params.Vout = Vout
    dab_params.PL = PL
    
    ipp, ZVS, ZCS, PL, pos, plot, modulation = optimize_mod_dab(Vin, Vout, PL, "EPS")
    if modulation == "EPS1":
        dab_params.D1, dab_params.D2 = float(pos[1]), 1.
    elif modulation == "EPS2":
        dab_params.D2, dab_params.D1 = float(pos[1]), 1.
    
    response = f"""The optimized modulation parameters for the EPS modulation under PL = {PL:.1f} W, 
    Vin = {Vin:.1f} V, Vout = {Vout:.1f} V are: {[item.round(3) for item in pos]}. The evaluated 
    peak-to-peak current is {ipp:.2f} A, number of ZVS switches is: {ZVS}, number of ZCS switches is {ZCS}."""
    return response, plot


def circuit_simulation_tool():    # chat_engine, prompt, messages_history
    """
        Design Task 2: Invoke/call/interface circuit simulation for DAB converters
    """
    print("Executing Task circuit_simulation_tool.")
    
    model_name = "DAB"
    file_path = os.path.abspath(f"../core/simulation/{model_name}.plecs")
    open_plecs(file_path)
    
    kwargs = {"Vin": dab_params.Vin, "Vref": dab_params.Vout, "P": dab_params.PL, 
              "D1": dab_params.D1, "D2": dab_params.D2, 
              "Ro": dab_params.Vout**2/dab_params.PL, "phi1": 0., "phi2": 0.,}
    print(kwargs)
    # conduct the plecs simulation
    thread = PlecsThread(model_name, file_path, **kwargs)
    thread.start()
    
    response = "Plecs simulation for designed DAB converters is conducting. Please wait a while..."
    return response
    
    
def multi_physics_tool():
    """
        Design Task 3: Invoke/call/interface multi-physics (electrical, magnetic, 
        thermal, mechanical) simulation of designed components.
    """
    print("Executing Task multi_physics_tool.")


conduct_design_tool = FunctionTool.from_defaults(fn=conduct_design_tool)
circuit_simulation_tool = FunctionTool.from_defaults(fn=circuit_simulation_tool)
multi_physics_tool = FunctionTool.from_defaults(fn=multi_physics_tool)


In [12]:
from dataclasses import dataclass


# define a class to store some key hyperparameters of DAB converters 
# Pass the parameters to simulation 
@dataclass
class DAB_Params:
    Vin: float = 0.
    Vout: float = 0.
    PL: float = 0.
    phi1: float = 0.
    phi2: float = 0.
    D1: float = 0.
    D2: float = 0.

dab_params = DAB_Params()
print(dab_params)


DAB_Params(Vin=0.0, Vout=0.0, PL=0.0, phi1=0.0, phi2=0.0, D1=0.0, D2=0.0)


In [13]:
# Create a simplified PE-GPT agent with FunctionTool routing capability
# define task_agent which routes the user query to different tools
def task_agent(llm_model: str, tools: List[FunctionTool]):
    """
        Define an LLM agent to judge and keep track of the design stage/task/workflow
    """

    llm = OpenAI(model=llm_model)
    agent = OpenAIAgent.from_tools(
        tools, llm=llm, verbose=True)
    
    return agent


llm_model = "gpt-4o-mini" # "gpt-4-0125-preview" # "gpt-3.5-turbo" 
tools = [conduct_design_tool,
         circuit_simulation_tool,
         multi_physics_tool]
llm = task_agent(llm_model, tools)

# Query for conducting modulation optimization
query = """Please help me design the EPS modulation for DAB converters.
    My Vi is 0.3 kv, output voltage is 360 V, load power is 200 W."""


response = llm.chat(f"""
Please invoke corresponding function based on user query.
Note that only one function will be called.
---------------------------------
User Query: {query}""")

print(dab_params)
if tkinter_enabled:
    show_popup(query, response)
else:
    print("*"*81)
    print("The user query is given below:")
    print(query)
    print("*"*81)
    print("The response is given below:")
    print(response)

Added user message to memory: 
Please invoke corresponding function based on user query.
Note that only one function will be called.
---------------------------------
User Query: Please help me design the EPS modulation for DAB converters.
    My Vi is 0.3 kv, output voltage is 360 V, load power is 200 W.


2025-03-17 18:14:41,753 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


=== Calling Function ===
Calling function: conduct_design_tool with args: {"Vin":300,"Vout":360,"PL":200}


2025-03-17 18:14:42.510 
  command:

    streamlit run D:\Anaconda\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-03-17 18:14:42,560 - pyswarms.single.global_best - INFO - Optimize for 50 iters with {'c1': 2.05, 'c2': 2.05, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████████████████████████████████████████████████████████|50/50, best_cost=-6.66
2025-03-17 18:15:28,071 - pyswarms.single.global_best - INFO - Optimization finished | best cost: -6.663459777832031, best pos: [-0.15702949  0.66200147]


Got output: ('The optimized modulation parameters for the EPS modulation under PL = 200.0 W, \n    Vin = 300.0 V, Vout = 360.0 V are: [-0.157, 0.662]. The evaluated \n    peak-to-peak current is 10.20 A, number of ZVS switches is: 8.0, number of ZCS switches is 0.0.', <_io.BytesIO object at 0x000001E60C164F90>)



2025-03-17 18:15:31,364 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


DAB_Params(Vin=300, Vout=360, PL=200, phi1=0.0, phi2=0.0, D1=1.0, D2=0.662)


In [14]:
from PIL import Image
import io

# Assuming you already have a BytesIO object named `buffer` with a plot saved in it
# buffer.seek(0) ensures you're at the start of the BytesIO stream
# buffer.seek(0)

# Open the image using PIL
image = Image.open(response.sources[0].raw_output[1])

# Display the image
image.show()

In [15]:
# Query for verifying the designed modulation in simulation
query = "Can you simulate the designed modulation for DAB converter?"
# query = "Can you simulate the temperature distribution field?"

response = llm.chat(f"""
Please invoke corresponding function based on user query.
Note that only one function will be called.
---------------------------------
User Query: {query}""")

if tkinter_enabled:
    show_popup(query, response)
else:
    print("*"*81)
    print("The user query is given below:")
    print(query)
    print("*"*81)
    print("The response is given below:")
    print(response)

Added user message to memory: 
Please invoke corresponding function based on user query.
Note that only one function will be called.
---------------------------------
User Query: Can you simulate the designed modulation for DAB converter?


2025-03-17 18:15:44,218 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


=== Calling Function ===
Calling function: circuit_simulation_tool with args: {}
Executing Task circuit_simulation_tool.
{'Vin': 300, 'Vref': 360, 'P': 200, 'D1': 1.0, 'D2': 0.662, 'Ro': 648.0, 'phi1': 0.0, 'phi2': 0.0}
Got output: Plecs simulation for designed DAB converters is conducting. Please wait a while...



2025-03-17 18:15:45,458 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
